### Analysis of Bias Frames from the new Shane Guide Camera

I figure here I should give a high-level report here.

This code runs the same analysis 4 times, one for each of the possible read settings on the conventional amplifier in the Andor iXon Ultra camera. The analysis is on the raw data from 27 bias frames. It takes 5 random frames and calculates the mean, median, and standard deviation of each frame, both with and without sigma clipping. 

It then stacks the 27 frames, and calculates sigma-clipped stats pixel by pixel. The resulant mean is saved as a "master bias frame".

After the frame analysis, it measures read noise. The code grabs a random frame, and displays it in DS9, waiting for the user to select a region of data. Once a region has been selected, it pulls in the data from that region, and takes statistics both with and without sigma clipping. The sigma-clipped standard deviation is taken as the read noise.

Since I don't have a measured gain value, these numbers are all in ADC. However, Andor provided some gain values. Using those, I can give some initial results.
|Settings      |Readnoise     |(Andor-provied value)|
|0.1MHz, gain 2: |3.58adc/2.83e- |(2.88e-)|
|0.1MHz, gain 1: |1.43adc/4.75e- |(4.88e-)|
|1.0Mhz, gain 2: |5.92adc/4.68e- |(4.72e-)|
|1.0Mhz, gain 1: |1.89adc/6.24e- |(6.30e-)|

Keep in mind this is using Andor's measurement of gain, and these are only from one frame each. 

In [1]:
from ccd_tools import *


In [18]:
# 0.1MHz Gain 2: least sensitive preamp gain, slowest read speed
path = '/home/lee/Data/bias_gain2_100kHz_20190628'
# match start of string, then look ahead infinitely to see if 'master' is not in string
pattern = re.compile(r'^(?!.*(master))')
filenames = get_filenames(path=path, pattern=pattern, extension='fits')

# do stats on the first 5 files
bias_stats = []
bias_clipped_stats = []
for file in filenames[:5]:
    with fits.open(path+'/'+file) as hdul:
        bias_stats.append(image_stats(hdul[0].data))
        bias_clipped_stats.append(image_stats(hdul[0].data, sigma_clip=True, sigma=4.0))

# print the unclipped bias results
print('bias frame stats:')
print('mean    |median  |std dev   ')
for stattuple in bias_stats:
    print(f'{stattuple[0]:.3f}', f'|{stattuple[1]:.3f}', f'|{stattuple[2]:.3f}')

# print the bias results, this time the clipped stuff
print('sigma clipped bias frame stats:')
print('mean    |median  |std dev   ')
for stattuple in bias_clipped_stats:
    print(f'{stattuple[0]:.3f}', f'|{stattuple[1]:.3f}', f'|{stattuple[2]:.3f}')

stacked_bias_mean, stacked_bias_median, stacked_bias_stddev = \
    sigma_clipped_frame_stats(filenames, path=path, sigma=4.0)

print('mean pixel count of the master bias frame:', f'{np.mean(stacked_bias_mean):.3f}')
print('median pixel count of same:', f'{np.median(stacked_bias_mean):.3f}')
print('standard deviation of same:', f'{np.std(stacked_bias_mean):.3f}')

bias frame stats:
mean    |median  |std dev   
500.397 |500.000 |3.585
500.560 |501.000 |3.582
500.619 |501.000 |3.591
499.734 |500.000 |3.587
499.390 |499.000 |3.587
sigma clipped bias frame stats:
mean    |median  |std dev   
500.397 |500.000 |3.583
500.561 |501.000 |3.580
500.619 |501.000 |3.588
499.734 |500.000 |3.586
499.389 |499.000 |3.585
mean pixel count of the master bias frame: 500.089
median pixel count of same: 500.107
standard deviation of same: 0.697


In [19]:
# now it's time to measure the read noise
# access an already open instance of ds9
ds9 = pyds9.DS9(target='bias-display')
pyds9.ds9_targets()
# open one of the files. the filename list is semi-random, so just grab the first entry
ds9.set('fits '+path+'/'+filenames[0])
input('Pause while you select data. Press enter to continue')
# in DS9, make a box region that avoids the edges of the image, to avoid any sort of trail-off or other edge
# weirdness. Also make sure no cosmic rays are in it. Then select the box.
bias_data = get_ds9_region(ds9).data
# calculate statistics on the data, such that it prints the results
print('readnoise for 100kHz, gain 2')
print('Statistics without clipping')
image_stats(bias_data, sigma_clip=False, verbose=True)
# readnoise is the std dev of the data, after sigma clipping
print('------------------------------------')
print('Statistics after clipping to 4-sigma')
_ , _, readnoise = image_stats(bias_data, sigma_clip=True, sigma=4.0, verbose=True)
print('------------------------------------')
print('read noise:', readnoise)

Pause while you select data. Press enter to continue


/home/lee/PycharmProjects/pyds9-implementation/ccd_tools.py:1142: UserWarning: "Keyword 'DATASEC' not found." Unable to validate region.
Region may exceed image bounds
  
/home/lee/PycharmProjects/pyds9-implementation/ccd_tools.py:1159: UserWarning: "Keyword 'BIASSEC' not found." Unable to perform bias subtraction.
  warnings.warn(message, category=UserWarning)


readnoise for 100kHz, gain 2
Statistics without clipping
Min pixel value: 482.00
Max pixel value: 517.00
Mean: 500.42
Median: 500.00
Std: 3.58
------------------------------------
Statistics after clipping to 4-sigma
Min pixel value: 482.00
Max pixel value: 517.00
Mean: 500.42
Median: 500.00
Std: 3.58
------------------------------------
read noise: 3.5817127


In [20]:
# 0.1MHz Gain 1: More sensitive preamp gain (read: higher gain), slowest read speed
path = '/home/lee/Data/bias_gain1_100kHz_20190628'
# match start of string, then look ahead infinitely to see if 'master' is not in string
pattern = re.compile(r'^(?!.*(master))')
filenames = get_filenames(path=path, pattern=pattern, extension='fits')

# do stats on the first 5 files
bias_stats = []
bias_clipped_stats = []
for file in filenames[:5]:
    with fits.open(path+'/'+file) as hdul:
        bias_stats.append(image_stats(hdul[0].data))
        bias_clipped_stats.append(image_stats(hdul[0].data, sigma_clip=True, sigma=4.0))

# print the unclipped bias results
print('bias frame stats:')
print('mean    |median  |std dev   ')
for stattuple in bias_stats:
    print(f'{stattuple[0]:.3f}', f'|{stattuple[1]:.3f}', f'|{stattuple[2]:.3f}')

# print the bias results, this time the clipped stuff
print('sigma clipped bias frame stats:')
print('mean    |median  |std dev   ')
for stattuple in bias_clipped_stats:
    print(f'{stattuple[0]:.3f}', f'|{stattuple[1]:.3f}', f'|{stattuple[2]:.3f}')

stacked_bias_mean, stacked_bias_median, stacked_bias_stddev = \
    sigma_clipped_frame_stats(filenames, path=path, sigma=4.0)

print('mean pixel count of the master bias frame:', f'{np.mean(stacked_bias_mean):.3f}')
print('median pixel count of same:', f'{np.median(stacked_bias_mean):.3f}')
print('standard deviation of same:', f'{np.std(stacked_bias_mean):.3f}')

bias frame stats:
mean    |median  |std dev   
500.109 |500.000 |1.435
500.180 |500.000 |1.434
500.171 |500.000 |1.435
500.166 |500.000 |1.435
500.170 |500.000 |1.444
sigma clipped bias frame stats:
mean    |median  |std dev   
500.109 |500.000 |1.434
500.180 |500.000 |1.433
500.171 |500.000 |1.434
500.166 |500.000 |1.434
500.170 |500.000 |1.443
mean pixel count of the master bias frame: 500.178
median pixel count of same: 500.179
standard deviation of same: 0.273


In [21]:
# read noise for 100kHz, gain 1
# access an already open instance of ds9
ds9 = pyds9.DS9(target='bias-display')
pyds9.ds9_targets()
# open one of the files. the filename list is semi-random, so just grab the first entry
ds9.set('fits '+path+'/'+filenames[0])
input('Pause while you select data. Press enter to continue')
# in DS9, make a box region that avoids the edges of the image, to avoid any sort of trail-off or other edge
# weirdness. Also make sure no cosmic rays are in it. Then select the box.
bias_data = get_ds9_region(ds9).data
# calculate statistics on the data, such that it prints the results
print('readnoise for 100kHz, gain 1')
print('Statistics without clipping')
image_stats(bias_data, sigma_clip=False, verbose=True)
# readnoise is the std dev of the data, after sigma clipping
print('------------------------------------')
print('Statistics after clipping to 4-sigma')
_ , _, readnoise = image_stats(bias_data, sigma_clip=True, sigma=4.0, verbose=True)
print('------------------------------------')
print('read noise:', readnoise)

Pause while you select data. Press enter to continue


/home/lee/PycharmProjects/pyds9-implementation/ccd_tools.py:1142: UserWarning: "Keyword 'DATASEC' not found." Unable to validate region.
Region may exceed image bounds
  
/home/lee/PycharmProjects/pyds9-implementation/ccd_tools.py:1159: UserWarning: "Keyword 'BIASSEC' not found." Unable to perform bias subtraction.
  warnings.warn(message, category=UserWarning)


readnoise for 100kHz, gain 1
Statistics without clipping
Min pixel value: 494.00
Max pixel value: 506.00
Mean: 500.11
Median: 500.00
Std: 1.44
------------------------------------
Statistics after clipping to 4-sigma
Min pixel value: 494.00
Max pixel value: 506.00
Mean: 500.11
Median: 500.00
Std: 1.43
------------------------------------
read noise: 1.4339929


In [22]:
# 1.0MHz Gain 2:  lower gain, fastest read speed for conventional amplifier
path = '/home/lee/Data/bias_gain2_1MHz_20190703'
# match start of string, then look ahead infinitely to see if 'master' is not in string
pattern = re.compile(r'^(?!.*(master))')
filenames = get_filenames(path=path, pattern=pattern, extension='fits')

# do stats on the first 5 files
bias_stats = []
bias_clipped_stats = []
for file in filenames[:5]:
    with fits.open(path+'/'+file) as hdul:
        bias_stats.append(image_stats(hdul[0].data))
        bias_clipped_stats.append(image_stats(hdul[0].data, sigma_clip=True, sigma=4.0))

# print the unclipped bias results
print('bias frame stats:')
print('mean    |median  |std dev   ')
for stattuple in bias_stats:
    print(f'{stattuple[0]:.3f}', f'|{stattuple[1]:.3f}', f'|{stattuple[2]:.3f}')

# print the bias results, this time the clipped stuff
print('sigma clipped bias frame stats:')
print('mean    |median  |std dev   ')
for stattuple in bias_clipped_stats:
    print(f'{stattuple[0]:.3f}', f'|{stattuple[1]:.3f}', f'|{stattuple[2]:.3f}')

stacked_bias_mean, stacked_bias_median, stacked_bias_stddev = \
    sigma_clipped_frame_stats(filenames, path=path, sigma=4.0)

print('mean pixel count of the master bias frame:', f'{np.mean(stacked_bias_mean):.3f}')
print('median pixel count of same:', f'{np.median(stacked_bias_mean):.3f}')
print('standard deviation of same:', f'{np.std(stacked_bias_mean):.3f}')

bias frame stats:
mean    |median  |std dev   
500.213 |500.000 |5.928
501.224 |501.000 |6.031
501.205 |501.000 |5.925
500.210 |500.000 |5.923
500.229 |500.000 |5.917
sigma clipped bias frame stats:
mean    |median  |std dev   
500.212 |500.000 |5.924
501.221 |501.000 |5.923
501.205 |501.000 |5.921
500.209 |500.000 |5.920
500.228 |500.000 |5.914
mean pixel count of the master bias frame: 500.323
median pixel count of same: 500.321
standard deviation of same: 1.123


In [24]:
# read noise for 1MHz, gain 2
# access an already open instance of ds9
ds9 = pyds9.DS9(target='bias-display')
pyds9.ds9_targets()
# open one of the files. the filename list is semi-random, so just grab the first entry
ds9.set('fits '+path+'/'+filenames[0])
input('Pause while you select data. Press enter to continue')
# in DS9, make a box region that avoids the edges of the image, to avoid any sort of trail-off or other edge
# weirdness. Also make sure no cosmic rays are in it. Then select the box.
bias_data = get_ds9_region(ds9).data
# calculate statistics on the data, such that it prints the results
print('readnoise for 1MHz, gain 2')
print('Statistics without clipping')
image_stats(bias_data, sigma_clip=False, verbose=True)
# readnoise is the std dev of the data, after sigma clipping
print('------------------------------------')
print('Statistics after clipping to 4-sigma')
_ , _, readnoise = image_stats(bias_data, sigma_clip=True, sigma=4.0, verbose=True)
print('------------------------------------')
print('read noise:', readnoise)

Pause while you select data. Press enter to continue


/home/lee/PycharmProjects/pyds9-implementation/ccd_tools.py:1142: UserWarning: "Keyword 'DATASEC' not found." Unable to validate region.
Region may exceed image bounds
  
/home/lee/PycharmProjects/pyds9-implementation/ccd_tools.py:1159: UserWarning: "Keyword 'BIASSEC' not found." Unable to perform bias subtraction.
  warnings.warn(message, category=UserWarning)


readnoise for 1MHz, gain 2
Statistics without clipping
Min pixel value: 473.00
Max pixel value: 531.00
Mean: 500.20
Median: 500.00
Std: 5.92
------------------------------------
Statistics after clipping to 4-sigma
Min pixel value: 473.00
Max pixel value: 531.00
Mean: 500.20
Median: 500.00
Std: 5.92
------------------------------------
read noise: 5.916281


In [26]:
# 1.0MHz Gain 1:  higher gain, fastest read speed for conventional amplifier
path = '/home/lee/Data/bias_gain1_1MHz_20190703'
# match start of string, then look ahead infinitely to see if 'master' is not in string
pattern = re.compile(r'^(?!.*(master))')
filenames = get_filenames(path=path, pattern=pattern, extension='fits')

# do stats on the first 5 files
bias_stats = []
bias_clipped_stats = []
for file in filenames[:5]:
    with fits.open(path+'/'+file) as hdul:
        bias_stats.append(image_stats(hdul[0].data))
        bias_clipped_stats.append(image_stats(hdul[0].data, sigma_clip=True, sigma=4.0))

# print the unclipped bias results
print('bias frame stats:')
print('mean    |median  |std dev   ')
for stattuple in bias_stats:
    print(f'{stattuple[0]:.3f}', f'|{stattuple[1]:.3f}', f'|{stattuple[2]:.3f}')

# print the bias results, this time the clipped stuff
print('sigma clipped bias frame stats:')
print('mean    |median  |std dev   ')
for stattuple in bias_clipped_stats:
    print(f'{stattuple[0]:.3f}', f'|{stattuple[1]:.3f}', f'|{stattuple[2]:.3f}')

stacked_bias_mean, stacked_bias_median, stacked_bias_stddev = \
    sigma_clipped_frame_stats(filenames, path=path, sigma=4.0)

print('mean pixel count of the master bias frame:', f'{np.mean(stacked_bias_mean):.3f}')
print('median pixel count of same:', f'{np.median(stacked_bias_mean):.3f}')
print('standard deviation of same:', f'{np.std(stacked_bias_mean):.3f}')

bias frame stats:
mean    |median  |std dev   
500.367 |500.000 |2.013
500.417 |500.000 |1.901
500.427 |500.000 |1.902
500.425 |500.000 |1.901
500.389 |500.000 |1.900
sigma clipped bias frame stats:
mean    |median  |std dev   
500.365 |500.000 |1.897
500.417 |500.000 |1.900
500.427 |500.000 |1.900
500.425 |500.000 |1.900
500.389 |500.000 |1.899
mean pixel count of the master bias frame: 500.258
median pixel count of same: 500.250
standard deviation of same: 0.360


In [27]:
# read noise for 1MHz, gain 1
# access an already open instance of ds9
ds9 = pyds9.DS9(target='bias-display')
pyds9.ds9_targets()
# open one of the files. the filename list is semi-random, so just grab the first entry
ds9.set('fits '+path+'/'+filenames[0])
input('Pause while you select data. Press enter to continue')
# in DS9, make a box region that avoids the edges of the image, to avoid any sort of trail-off or other edge
# weirdness. Also make sure no cosmic rays are in it. Then select the box.
bias_data = get_ds9_region(ds9).data
# calculate statistics on the data, such that it prints the results
print('readnoise for 1MHz, gain 1')
print('Statistics without clipping')
image_stats(bias_data, sigma_clip=False, verbose=True)
# readnoise is the std dev of the data, after sigma clipping
print('------------------------------------')
print('Statistics after clipping to 4-sigma')
_ , _, readnoise = image_stats(bias_data, sigma_clip=True, sigma=4.0, verbose=True)
print('------------------------------------')
print('read noise:', readnoise)

Pause while you select data. Press enter to continue


/home/lee/PycharmProjects/pyds9-implementation/ccd_tools.py:1142: UserWarning: "Keyword 'DATASEC' not found." Unable to validate region.
Region may exceed image bounds
  
/home/lee/PycharmProjects/pyds9-implementation/ccd_tools.py:1159: UserWarning: "Keyword 'BIASSEC' not found." Unable to perform bias subtraction.
  warnings.warn(message, category=UserWarning)


readnoise for 1MHz, gain 1
Statistics without clipping
Min pixel value: 492.00
Max pixel value: 509.00
Mean: 500.37
Median: 500.00
Std: 1.90
------------------------------------
Statistics after clipping to 4-sigma
Min pixel value: 492.00
Max pixel value: 509.00
Mean: 500.37
Median: 500.00
Std: 1.89
------------------------------------
read noise: 1.8943143
